In [ ]:
import pandas as pd
import numpy as np
import json
import os
import matplotlib.pyplot as plt
from pathlib import Path
from pandas_path import path

import rasterio

import warnings
warnings.filterwarnings("ignore")

In [ ]:
# This is where our downloaded images and metadata live locally
DATA_PATH = Path.cwd().parent / "data" / "raw" / "train_features"

In [ ]:
train_metadata = pd.read_csv(
    DATA_PATH / "flood-training-metadata.csv", parse_dates=["scene_start"]
)

In [ ]:
train_metadata.head()

In [ ]:
train_metadata.shape

In [ ]:
train_metadata.chip_id.nunique()

We have 542 unique chip id, every id has two images _vh and _vv

In [ ]:
train_metadata["feature_path"] = (
    str(DATA_PATH / "train_features")
    / train_metadata.image_id.path.with_suffix(".tif").path
)

train_metadata["label_path"] = (
    str(DATA_PATH / "train_labels")
    / train_metadata.chip_id.path.with_suffix(".tif").path
)

In [ ]:
train_metadata

In [ ]:
# Examine an arbitrary image
image_path = train_metadata.feature_path[0]
with rasterio.open(image_path) as img:
    metadata = img.meta
    bounds = img.bounds
    data = img.read(1)  # read a single band
    
with rasterio.open(image_path) as img:
    gdal_mask = img.dataset_mask()
    
with rasterio.open(image_path) as img:
    numpy_mask = img.read(1, masked=True)
    
f, ax = plt.subplots(1, 2, figsize=(9, 9))
ax[0].imshow(gdal_mask)
ax[1].imshow(numpy_mask)
plt.show()

In [ ]:
# Helper functions for visualizing Sentinel-1 images
def scale_img(matrix):
    """
    Returns a scaled (H, W, D) image that is visually inspectable.
    Image is linearly scaled between min_ and max_value, by channel.

    Args:
        matrix (np.array): (H, W, D) image to be scaled

    Returns:
        np.array: Image (H, W, 3) ready for visualization
    """
    # Set min/max values
    min_values = np.array([-23, -28, 0.2])
    max_values = np.array([0, -5, 1])

    # Reshape matrix
    w, h, d = matrix.shape
    matrix = np.reshape(matrix, [w * h, d]).astype(np.float64)

    # Scale by min/max
    matrix = (matrix - min_values[None, :]) / (
        max_values[None, :] - min_values[None, :]
    )
    matrix = np.reshape(matrix, [w, h, d])

    # Limit values to 0/1 interval
    return matrix.clip(0, 1)

def numpy_mask(image_path):
    with rasterio.open(image_path) as img:
        metadata = img.meta
        bounds = img.bounds
        data = img.read(1)  # read a single band

    with rasterio.open(image_path) as img:
        return img.read(1, masked=True)
    
def gdal_mask(image_path):
    with rasterio.open(image_path) as img:
        metadata = img.meta
        bounds = img.bounds
        data = img.read(1)  # read a single band

    with rasterio.open(image_path) as img:
        return img.dataset_mask()
    

def create_false_color_composite(path_vv, path_vh):
    """
    Returns a S1 false color composite for visualization.

    Args:
        path_vv (str): path to the VV band
        path_vh (str): path to the VH band

    Returns:
        np.array: image (H, W, 3) ready for visualization
    """
    # Read VV/VH bands
    with rasterio.open(path_vv) as vv:
        vv_img = vv.read(1)
    with rasterio.open(path_vh) as vh:
        vh_img = vh.read(1)

    # Stack arrays along the last dimension
    s1_img = np.stack((vv_img, vh_img), axis=-1)

    # Create false color composite
    img = np.zeros((512, 512, 3), dtype=np.float32)
    img[:, :, :2] = s1_img.copy()
    img[:, :, 2] = s1_img[:, :, 0] / s1_img[:, :, 1]

    return scale_img(img)


def display_chip(row):
    """
    Plots a 3-channel representation of VV/VH polarizations as a single chip (image 1).
    Overlays a chip's corresponding water label (image 2).

    Args:
        random_state (int): random seed used to select a chip

    Returns:
        plot.show(): chip and labels plotted with pyplot
    """
    f, ax = plt.subplots(2, 2, figsize=(15, 15))

    # Select a random chip from train_metadata
    #random_chip = train_metadata.chip_id.sample(random_state=random_state).values[0]
    chip_df = train_metadata.iloc[row*2:row*2+2]

    # Extract paths to image files
    vv_path = chip_df[chip_df.polarization == "vv"].feature_path.values[0]
    vh_path = chip_df[chip_df.polarization == "vh"].feature_path.values[0]
    label_path = chip_df.label_path.values[0]

    # Create false color composite
    s1_img = create_false_color_composite(vv_path, vh_path)

    #visualize radar image
    ax[0][0].imshow(numpy_mask(vv_path))
    ax[0][0].set_title("Vertical-vertical band")
    ax[0][1].imshow(numpy_mask(vh_path))
    ax[0][1].set_title("Vertical-horizontal band")
    
    # Visualize features
    ax[1][0].imshow(s1_img, cmap="gray")
    ax[1][0].set_title("False color composite image", fontsize=14)

    # Load water mask
    with rasterio.open(label_path) as lp:
        lp_img = lp.read(1)

    # Mask missing data and 0s for visualization
    label = np.ma.masked_where((lp_img == 0) | (lp_img == 255), lp_img)

    #visualize answer
    #ax[1][1].imshow(s1_img)
    ax[1][1].imshow(label, cmap="cool")
    ax[1][1].set_title("Image with WaterLabel", fontsize=14)

    #plt.tight_layout(pad=5)
    plt.show()

In [ ]:
display_chip(3)

## Check if the data is well distributed between the flood events

In [ ]:
def count_pixel(path, value):
    with rasterio.open(path) as img:
        data = img.read(1)
    return (data == value).sum()

In [ ]:
#count the number of white, black and missing pixels in every image
# Examine an arbitrary image
for path in train_metadata['label_path'].to_list():
    train_metadata.loc[train_metadata['label_path'] == path, 'no_flood_%'] = count_pixel(path, 0)/(512*512)
    train_metadata.loc[train_metadata['label_path'] == path, 'flood_%'] = count_pixel(path, 1)/(512*512)
    train_metadata.loc[train_metadata['label_path'] == path, 'missing_%'] = count_pixel(path, 255)/(512*512)
    
train_metadata[['image_id', 'location', 'no_flood_%', 'flood_%', 'missing_%']]

In [ ]:
df = train_metadata.groupby('flood_id', as_index=False).agg(np.mean)
df.describe()

In [ ]:
import seaborn as sns
order = train_metadata.groupby('flood_id').agg(np.mean).index.to_list()
fig, ax = plt.subplots(4,1, figsize=(15,20))
sns.boxplot(x="flood_id", y="flood_%", data=train_metadata, order=order, ax=ax[0])
ax[0].set_title('flood_id vs flood_%');
sns.boxplot(x="flood_id", y="no_flood_%", data=train_metadata, order=order, ax=ax[1])
ax[1].set_title('flood_id vs no_flood_%');
sns.boxplot(x="flood_id", y="missing_%", data=train_metadata, order=order, ax=ax[2])
ax[2].set_title('flood_id vs missing_%');
sns.countplot(x="flood_id", data=train_metadata, order=order, ax=ax[3])
ax[1].set_title('Count of flood_id');
plt.show()